In [1]:
import numpy as np
import pandas as pd
import os

<h1>Daten Einlesen</h1>

In [2]:
unpickled_df = pd.read_pickle("../WI3_BusinessIntelligence_Data/data_inside_20211010_160005_one_hour_example.pkl")

print(unpickled_df)

{'accelerometer_x': array([-7.0458984, -7.0458984, -7.0654297, ..., -7.104492 , -7.104492 ,
       -7.060547 ], dtype=float32), 'accelerometer_y': array([6.9628906, 6.9628906, 6.982422 , ..., 6.9433594, 6.9433594,
       6.8896484], dtype=float32), 'accelerometer_z': array([0.7324219 , 0.7324219 , 0.7763672 , ..., 0.7910156 , 0.7910156 ,
       0.80078125], dtype=float32), 'magnetometer_x': array([-52250000., -54125000., -54125000., ..., -53437500., -54187500.,
       -54187500.], dtype=float32), 'magnetometer_y': array([4437500., -312500., -312500., ..., 1875000.,  750000.,  750000.],
      dtype=float32), 'magnetometer_z': array([-7750000., -4250000., -4250000., ..., -5875000., -3875000.,
       -3875000.], dtype=float32), 'audio': array([0.00689697, 0.00643921, 0.00540161, ..., 0.01193237, 0.01010132,
       0.01739502], dtype=float32), 'timer': array([      0,       1,       2, ..., 3599905, 3599905, 3599906],
      dtype=uint32)}


In [3]:
print(unpickled_df['timer'].max())

86277650


<h1>Audio Step für Smoothing festlegen</h1>

In [3]:
# to 20Hz
# Calculation:   audio_step  =  2100d/s  /  20Hz  =  105s

audio_step = int(2100 / 20) 

print('audio steps:', audio_step)

audio steps: 105


<h1>Indizes für den Split der Sensor Dateien finden</h1>

In [5]:
dict_no_audio = {}
for k in unpickled_df.keys():
    if k != 'audio':
        dict_no_audio[k] = unpickled_df[k]
     
print(dict_no_audio['accelerometer_x'].size, ' / ', dict_no_audio['magnetometer_x'].size, ' / ', dict_no_audio['timer'].size)
############################################
# das hier ist ist hoffentlich nur für das 1h example notwendig
dict_no_audio['timer'] = np.append(dict_no_audio['timer'], 3599906)
#print('after append: ', dict_no_audio['accelerometer_x'].size, ' / ', dict_no_audio['magnetometer_x'].size, ' / ', dict_no_audio['timer'].size)
############################################

df_no_audio = pd.DataFrame(dict_no_audio)
df_no_audio

7415239  /  7415239  /  7415238


,accelerometer_x,accelerometer_y,accelerometer_z,magnetometer_x,magnetometer_y,magnetometer_z,timer
0,-7.045898,6.962891,0.732422,-52250000.0,4437500.0,-7750000.0,0
1,-7.045898,6.962891,0.732422,-54125000.0,-312500.0,-4250000.0,1
2,-7.065430,6.982422,0.776367,-54125000.0,-312500.0,-4250000.0,2
3,-7.104492,6.962891,0.776367,-54125000.0,-312500.0,-4250000.0,2
4,-7.104492,6.962891,0.776367,-54125000.0,-312500.0,-4250000.0,2
...,...,...,...,...,...,...,...
7415234,-7.109375,6.948242,0.815430,-53437500.0,1875000.0,-5875000.0,3599905
7415235,-7.109375,6.948242,0.815430,-53437500.0,1875000.0,-5875000.0,3599905
7415236,-7.104492,6.943359,0.791016,-53437500.0,1875000.0,-5875000.0,3599905
7415237,-7.104492,6.943359,0.791016,-54187500.0,750000.0,-3875000.0,3599906


In [6]:
# range von timer values an denen aufgeteilt werden muss
# enthält immer die letzte ms die in einem Step sein soll
val_range = np.array(range(49,unpickled_df['timer'].max(),50))
print (val_range)
print (val_range.size)

[     49      99     149 ... 3599799 3599849 3599899]
71998


In [7]:
# für jede ms in val_range den index der folgenden ms bestimmen (dieser index wird in der split funktion später gebraucht)
i_split_arr = []
i = 0
for ms in val_range:
    i_split_arr.append(dict_no_audio['timer'].searchsorted(ms, side='right'))
#print(i_split_arr)

In [8]:
print(len(i_split_arr))

71998


https://stackoverflow.com/questions/41255215/pandas-find-first-occurrence

numpy searchsorted bei hourly example: ~100ms - warum dauert es hier so viel länger?
  - Lenge des 'i_split_arr' nach 58m und keyboard interupt sind ~16k einträge. Das 'hour example' hat 72k einträge in ~100ms

In [ ]:
# überprüfen ob der richtige Index für den entsprechende Timer Wert gefunden wurde
# bei timer sollte 50 stehen (50ms == 1/20sec)
df_no_audio.iloc[i_split_arr[0]]

In [ ]:
###################### nach MS ################################################
# erstellen eines leeren dictionaries
smt_dict = {}

for variable in unpickled_df.keys():
    if "accelerometer" in variable:
        # Array wird anhand der Indizes der MS gesplittet, mit NaN values auf die selbe lenge gebracht und gestacked
        ac_split_arr = np.split(unpickled_df[variable], i_split_arr)
        pad = len(max(ac_split_arr, key=len))
        pad_ac_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in ac_split_arr])
        stack_ac_split_arr = np.vstack(pad_ac_split_arr)
        
        # Zusammenfassen und in dictionary schreiben, NaN values werden ignoriert
        # maximum
        smt_dict[variable+'_max'] = np.nanmax(stack_ac_split_arr, axis = 1)
        # 95% quantile
        smt_dict[variable+'_95q'] = np.nanquantile(stack_ac_split_arr, 0.95, axis = 1)
        # minimum
        smt_dict[variable+'_min'] = np.nanmin(stack_ac_split_arr, axis = 1)
        # 5% quantile
        smt_dict[variable+'_05q'] = np.nanquantile(stack_ac_split_arr, 0.05, axis = 1)
        
    elif "magnetometer" in variable:
        # Array wird anhand der Indizes der MS gesplittet, mit NaN values auf die selbe lenge gebracht und gestacked
        ma_split_arr = np.split(unpickled_df[variable], i_split_arr)
        pad = len(max(ma_split_arr, key=len))
        pad_ma_split_arr = np.array([np.append(i, [[np.nan]*(pad-len(i))]) for i in ma_split_arr])
        stack_ma_split_arr = np.vstack(pad_ma_split_arr)
        
        # Zusammenfassen und in dictionary schreiben, NaN values werden ignoriert
        # mean
        smt_dict[variable+'_mean'] = np.nanmean(stack_ma_split_arr, axis = 1)
        # median
        smt_dict[variable+'_med'] = np.nanmedian(stack_ma_split_arr, axis = 1)
        
    elif "audio" in variable:
        au_split_arr = np.vstack(np.split(unpickled_df[variable], range(audio_step, unpickled_df['audio'].size, audio_step))[:-1])
        
        # Zusammenfassen und in DataFrame schreiben, NaN values werden ignoriert
        # maximum
        smt_dict[variable+'_max'] = np.amax(au_split_arr, axis = 1)
        # 95% quantile
        smt_dict[variable+'_95q'] = np.quantile(au_split_arr, 0.95, axis = 1)

        
print('sensor ', smt_dict['accelerometer_x_max'].size)
print('audio ', smt_dict['audio_max'].size)